<a href="https://colab.research.google.com/github/bpalani/blyss-genai-apps/blob/main/Testing_and_learning_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install virtualenv
!virtualenv myenv
!source myenv/bin/activate;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 9.6 MB/s eta 0:00:00
created virtual environment CPython3.11.12.final.0-64 in 2768ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.0.1, setuptools==78.1.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
#!pip install langchain-text-splitters langchain-postgres
!pip install langchain langchain_core langchain_community pypdf
!pip install langchain_text_splitters
!pip install langgraph
!pip install langchain_google_vertexai
#!pip install google-genai
!pip install chromadb
#!pip install langchain-core==0.1.23
#!pip install langchain==0.1.1
#!pip install langchain-google-genai==0.0.6
#!pip install langchain-community==0.0.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting

In [3]:
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
#from langchain.vectorstores import Chroma
from chromadb import chromadb

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "https://storage.googleapis.com/genai-capstone-project/2024ltr.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

15


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=20, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

477

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector_store = Chroma(
    collection_name="berkshire_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
# Load from disk
#vectorstore_disk = Chroma(
#                        persist_directory="./chroma_db",       # Directory of db
#                        embedding_function=embeddings   # Embedding model
#                   )
# Get the Retriever interface for the store to use later.
# When an unstructured query is given to a retriever it will return documents.
# Read more about retrievers in the following link.
# https://python.langchain.com/docs/modules/data_connection/retrievers/
#
# Since only 1 document is stored in the Chroma vector store, search_kwargs `k`
# is set to 1 to decrease the `k` value of chroma's similarity search from 4 to
# 1. If you don't pass this value, you will get a warning.
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# To configure model parameters use the `generation_config` parameter.
# eg. generation_config = {"temperature": 0.7, "topP": 0.8, "topK": 40}
# If you only want to set a custom temperature for the model use the
# "temperature" parameter directly.

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
generation_config = {"temperature": 0.1, "topP": 0.8, "topK": 40}

In [ ]:

# Prompt template to query Gemini
llm_prompt_template = """You are a researcher for answering user questions.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use as many sentences as possible to answer the user's question. Explain in detail if necessary.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] template="You are a researcher for answering user questions.\nUse the following context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse as many sentences as possible to answer the user's question. Explain in detail if necessary.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


In [ ]:
# Combine data from documents to readable string format.
#def format_docs(docs):
#    return "\n\n".join(doc.page_content for doc in docs)

# Create stuff documents chain using LCEL.
#
# This is called a chain because you are chaining together different elements
# with the LLM. In the following example, to create the stuff chain, you will
# combine the relevant context from the website data matching the question, the
# LLM model, and the output parser together like a chain using LCEL.
#
# The chain implements the following pipeline:
# 1. Extract the website data relevant to the question from the Chroma
#    vector store and save it to the variable `context`.
# 2. `RunnablePassthrough` option to provide `question` when invoking
#    the chain.
# 3. The `context` and `question` are then passed to the prompt where they
#    are populated in the respective variables.
# 4. This prompt is then passed to the LLM (`gemini-2.0-flash`).
# 5. Output from the LLM is passed through an output parser
#    to structure the model's response.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)


In [ ]:

rag_chain.invoke("How did Berkshire perform in 2024? Explain Berkshire's businesses in detail. What did they do? How did they perform in 2024?")

"In 2024, Berkshire Hathaway performed better than expected. The document mentions that Berkshire has 189 operating businesses, but it does not list them in detail or provide specific performance data for each one. Therefore, a comprehensive overview of each business's activities and individual performance in 2024 cannot be provided based on the context.\n"